Sure! Here's a detailed explanation of how to use the **Ollama** model locally in a markdown format:

---

# Using the Ollama Model Locally

Ollama is a tool that allows you to run large language models (LLMs) locally on your machine. The following guide will show you how to install, set up, and use the **Ollama** model on your local environment.

## 1. **Prerequisites**

Before using the Ollama model locally, ensure that you meet the following requirements:

- **Operating System**: Ollama supports macOS, Windows, and Linux.
- **Hardware**: Your machine should have sufficient RAM and a good CPU/GPU for running large models (depending on the model size).

## 2. **Install Ollama**

### a) **Download Ollama**

You need to download and install Ollama for your platform. Follow these instructions:

- **macOS**: Download the `.dmg` file from [Ollama's website](https://ollama.com/), then double-click to install.
- **Windows**: Download the `.exe` installer from [Ollama's website](https://ollama.com/), and follow the installation prompts.
- **Linux**: You can install Ollama by following the [Linux installation instructions](https://ollama.com/docs).

### b) **Check Installation**

Once installed, you can check if Ollama is working by running the following command in the terminal (or command prompt for Windows):

```bash
ollama --version
```

This will return the version number if the installation was successful.

# Ingesting PDF

In [68]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [69]:
local_path='files/DouglasModestoResume.pdf'

# Local PDF file uploads 
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print('upload a PDF file')
  

In [70]:
data[0].page_content

"DOUGLAS MODESTO Senior Software Engineer\n\n+5511966123225\n\ndouglasgomesmodesto@gmail.com\n\nhttp://www.linkedin.com/in/douglasmodesto\n\nBrazil\n\nSummary\n\nOver 14 years of experience in software development Successfully delivered complex projects for small, medium-sized and big companies. Expertise in upholding best programming practices Applies principles such as SOLID and Design Patterns for robust and scalable solutions Skill set includes proﬁciency in C#, .NET framework, SQL Server, RESTful APIs, and cloud technologies like AWS Adept at collaborating with diverse teams and effective communication with stakeholders at all levels Strong problem-solving abilities Adaptability to new technologies Consistently delivers high-quality results\n\nSkills\n\nC#(14 years), .Net Core(6 years), .Net 6+(2 years), Python(2 years), JavaScript(8 years), Html 5/CSS(8 years), Angular(8 years), Typescript(8 years), Git flow(5 years), Git(5 years), Design Patterns (9 years), SOLID Principles (9 y

# Vector Embedding

In [71]:
# Downloading the models from ollama

!ollama pull nomic-embed-text
!ollama pull qwen2.5:latest

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠴ pulling manifest 
pulling 2bada8a74506... 100% ▕████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b... 100% ▕████████████████▏   68 B                         
pulling eb4402837c78... 100% ▕████████████████▏ 1.5 KB                         
pulling 832dd9e00a68... 100% ▕████████████████▏  11 KB                         
pulling 2f15b3218f05... 100% ▕████████████████▏  487 B         

In [72]:
!ollama list

NAME                       ID              SIZE      MODIFIED      
qwen2.5:latest             845dbda0ea48    4.7 GB    2 seconds ago    
nomic-embed-text:latest    0a109f422b47    274 MB    2 seconds ago    


In [73]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [74]:
# Split and Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=75)
chunks = text_splitter.split_documents(data)

In [76]:
# Embedding 

embedding = OllamaEmbeddings(model='nomic-embed-text')


In [78]:
# Add to vector database

vector_db = Chroma.from_documents(
  documents=chunks,
  embedding=embedding,
  collection_name='local_rag'
)

# Retrieval

In [80]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.retrievers.multi_query import MultiQueryRetriever

In [81]:
# LLM from Ollama

local_model = 'qwen2.5:latest'
llm = ChatOllama(model=local_model)

In [82]:
QUERY_PROMPT = PromptTemplate(
  input_variables=['question'],
  template="""
    You are an AI language model assistant. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. 
    By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines.
    
    Original question: {question}
  """
)

In [83]:
retriever = MultiQueryRetriever.from_llm(
  vector_db.as_retriever(),
  llm,
  prompt=QUERY_PROMPT
)

# RAG prompt
template="""  Answer the question in portugues from Brazil based ONLY on the following context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template=template)

In [84]:
setup_and_retrieval = RunnableParallel(
  {
    'question': RunnablePassthrough() ,
    'context': retriever
  }
)

In [85]:
# Create the chain

chain = (
  setup_and_retrieval 
  | prompt 
  | llm 
  | StrOutputParser()
)



In [86]:
chain.invoke('Quais são os principais pontos sobre o Douglas')

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


"Douglas Modesto é um engenheiro de software sênior com mais de 14 anos de experiência no desenvolvimento de software. Ele tem uma vasta experiência em projetos complexos, tanto para pequenas quanto para grandes empresas, e se especializou em práticas de programação de ponta, como SOLID e padrões de design.\n\nAlguns dos seus principais habilidades incluem proficiência em C#, .NET Framework, SQL Server, APIs RESTful e tecnologias de nuvem como AWS. Ele possui uma forte capacidade de colaborar com equipes diversificadas e comunicar-se efetivamente com todos os níveis de stakeholders. Douglas é conhecido por sua resiliência técnica e habilidades problemáticas.\n\nAlguns dos seus projetos notáveis incluem a arquitetura de serviços baseados em nuvem e IA para um banco, a implementação de um sistema que facilita a comunicação entre consultores e clientes para uma empresa de investimentos, bem como a automação do processamento de dados utilizando RPA para uma instituição bancária.\n\nDouglas

"Douglas Modesto é um engenheiro de software sênior com mais de 14 anos de experiência no desenvolvimento de software. Ele tem uma vasta experiência em projetos complexos, tanto para pequenas quanto para grandes empresas, e se especializou em práticas de programação de ponta, como SOLID e padrões de design.\n\nAlguns dos seus principais habilidades incluem proficiência em C#, .NET Framework, SQL Server, APIs RESTful e tecnologias de nuvem como AWS. Ele possui uma forte capacidade de colaborar com equipes diversificadas e comunicar-se efetivamente com todos os níveis de stakeholders. Douglas é conhecido por sua resiliência técnica e habilidades problemáticas.\n\nAlguns dos seus projetos notáveis incluem a arquitetura de serviços baseados em nuvem e IA para um banco, a implementação de um sistema que facilita a comunicação entre consultores e clientes para uma empresa de investimentos, bem como a automação do processamento de dados utilizando RPA para uma instituição bancária.\n\nDouglas possui formação na FATEC – Universidade Tecnológica de São Paulo em Análise e Desenvolvimento de Sistemas, bem como MBA's em Inteligência Artificial & Machine Learning pela FIAP Administração e Paulista IT University, e em Arquitetura de Software Distribuído pela PUC Minas. Seu idioma nativo é o português e ele possui um nível B2-C1 de inglês.\n\nAs principais tecnologias que ele utiliza incluem .NET 8, C#, Python, Angular, TypeScript, Git Flow, Git, Design Patterns, SQL Server, MongoDB, CQRS, RabbitMQ, Kafka, AWS Cloud, Microservices, Event-Oriented Architecture e Docker."

In [36]:
# Delete all collection in the db
vector_db.delete_collection()